### **"G" Index** for calculating geographical scattering of votes

"[...] the index deducts the votes observed for a given deputy, in a given municipality, from the vote that would be expected for that same deputy, in the same municipality, if the votes were randomly distributed, or proportional to the number of votres among the municipalities. An intuitive way to understand this index is by observing that if a deputy had his votes distributed exactly according to the distribution if voters among the state's municipalities, the value of his 'G' index would be equal to zero" (AVELINO; BIDERMAN; SILVA, 2016, pp. 1100-1101. Free translation.).

$$ G_d = \sum_m \left( \frac{V_{dm}}{V_d} - \frac{V_m}{V} \right)^2$$

Where:

$V_{dm}$ is the number of votes for deputy $d$ in the municipality $m$

$V_d \equiv \sum_m v_{dm}$ is the total of votes for deputy $d$ statewide

$V_m \equiv \sum_d v_{dm}$ is the total of votes for deputies in each municipality

$V \equiv \sum_d \sum_m V_{dm}$ is the total of votes for deputies statewide

"For example, it is expected that any candidate in Amapá will obtain almost 60% of votes in the municipality of Macapá, since almost 60% of the state's electorate is concentrated in that municipality. If, on the other hand, a candidate adopted a strategy of massively concentrating his resources in Macapá, he would be expected to obtain a proportion of votes greater than 60% in that municipality. **The 'G' index therefore captures the results of candidates' electoral investments in certain areas, investments that would determine the concentration of votes.**" (AVELINO; BIDERMAN; SILVA, 2016, pp. 1101. Emphasis mine. Free translation.).

In [226]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
import basedosdados as bd
from shapely import wkt

In [227]:
query = """
SELECT DISTINCT id_candidato_bd, ano
FROM basedosdados.br_tse_eleicoes.resultados_candidato
WHERE ano = 2022
AND resultado IN ('eleito', 'eleito por qp')
AND sigla_uf = 'RJ'
AND id_candidato_bd 
IN (SELECT DISTINCT id_candidato_bd 
FROM basedosdados.br_tse_eleicoes.resultados_candidato
WHERE ano = 2018)
"""

elected = bd.read_sql(query, billing_project_id="diamundialdainfancia")

Downloading: 100%|██████████| 67/67 [00:00<00:00, 204.20rows/s]


In [228]:
#get random element from a column
random = elected['id_candidato_bd'].sample().iloc[0]
print(f'The random congressperson has id {random}.')

The random congressperson has id 1476513.


In [229]:
query = f"""
SELECT DISTINCT resultado.ano, resultado.sigla_uf, resultado.id_municipio, resultado.sequencial_candidato, resultado.id_candidato_bd, candidato.nome, candidato.sigla_partido, candidato.numero, sum(resultado.votos) as votos
FROM basedosdados.br_tse_eleicoes.resultados_candidato_secao resultado
INNER JOIN basedosdados.br_tse_eleicoes.candidatos candidato
ON resultado.id_candidato_bd = candidato.id_candidato_bd
AND resultado.ano = candidato.ano
WHERE resultado.ano in (2018, 2022)
AND resultado.sigla_uf = 'RJ'
AND candidato.id_candidato_bd = '{random}'
GROUP BY resultado.ano, resultado.sigla_uf, resultado.id_municipio, resultado.sequencial_candidato, resultado.id_candidato_bd, candidato.nome, candidato.sigla_partido, candidato.numero
"""

V_dm = bd.read_sql(query, billing_project_id="diamundialdainfancia")

Downloading: 100%|██████████| 184/184 [00:00<00:00, 529.38rows/s]


In [230]:
print(f'The name of the congressperson is {V_dm["nome"].iloc[0]}, who ran for the party {V_dm["sigla_partido"].iloc[0]}.')

The name of the congressperson is Helio Fernando Barbosa Lopes, who ran for the party PSL.


In [231]:
# V_dm = each line on the dataframe
V_d = V_dm.groupby(['ano', 'sigla_uf'])['votos'].sum().reset_index(name = 'votes for congressperson statewide')

In [232]:
query = """
SELECT ano, sigla_uf, id_municipio, cargo, SUM(votos_validos) as votos_deputados
FROM basedosdados.br_tse_eleicoes.detalhes_votacao_municipio
WHERE ano in (2018, 2022)
AND sigla_uf = 'RJ'
AND cargo = 'deputado estadual'
GROUP BY ano, sigla_uf, id_municipio, cargo
"""

V_m = bd.read_sql(query, billing_project_id="diamundialdainfancia")

Downloading: 100%|██████████| 184/184 [00:00<00:00, 484.03rows/s]


In [233]:
V_m.rename(columns = {'votos_deputados': 'votes for deputies on municipality'}, inplace=True)

In [234]:
V = V_m.groupby(['ano', 'sigla_uf'])['votes for deputies on municipality'].sum().reset_index(name = 'votes for deputies statewide')

In [235]:
merged = V_dm.merge(V_d, on=['ano', 'sigla_uf'], how='left') 
merged = merged.merge(V_m, on = ['ano', 'id_municipio'], how = 'left')

merged.rename(columns = {'sigla_uf_x': 'sigla_uf', 'votos': 'votes for congressperson on municipality'}, inplace = True)

merged = merged.merge(V, on = ['ano', 'sigla_uf'], how = 'left')

In [236]:
df = merged[['ano', 'sigla_uf', 'id_municipio', 'nome', 'sigla_partido', 'numero', 'votes for congressperson on municipality', 'votes for congressperson statewide', 'votes for deputies on municipality', 'votes for deputies statewide']]

In [237]:
df['G index'] = ((df['votes for congressperson on municipality'] / df['votes for congressperson statewide']) - (df['votes for deputies on municipality'] / df['votes for deputies statewide']))**2 
df.groupby('ano')['G index'].sum().reset_index(name = 'G index')

,ano,G index
0,2018,0.017844
1,2022,0.006606


The G index indicates the level of concentration. Graphically, we present the **Locational Quotient (LQ)**.

### **Locational Quotient (LQ)** for calculating geographical scattering of votes

"[...] the LQ of a candidate per municipality is determined by the ratio between two proportions: **the proportion of votes obtained in the municipality in relation to the candidate's total vote, and the proportion of voters from the municipality in the state electorate, that is, the candidate's expected vote in that municipality.** Easy to read, when the LQ is of value one, this indicates that the candidate obtained the vote that would be expected in that municipality, if the votes were randomly distributed. LQs below one indicate a lower-than-expected vote, and LQs above one indicate a higher-than-expected vote." (AVELINO; BIDERMAN; SILVA, 2016, pp. 1101-1102. Emphasis mine. Free translation).

$$LQ = \frac{V_{dm}}{V_m} \times \frac{E}{E_m}$$

Where:

$E_m$ is the number of voters in each municipality $m$

$E \equiv \sum_m e_m$ is the number of voters statewide

$V_{dm}$ is the number of votes for congressperson $d$ in municipality $m$

$V_m \equiv \sum_d v_{dm}$ is the total number of votes for deputies in the municipality

In [238]:
query = """
SELECT DISTINCT ano, sigla_uf, id_municipio, aptos, cargo
FROM basedosdados.br_tse_eleicoes.detalhes_votacao_municipio
WHERE ano in (2018, 2022)
AND sigla_uf = 'RJ'
AND cargo = 'deputado estadual'
"""

E_m = bd.read_sql(query, billing_project_id="diamundialdainfancia")

Downloading: 100%|██████████| 184/184 [00:00<00:00, 517.91rows/s]


In [239]:
E_m.rename(columns = {'aptos': 'voters per municipality'}, inplace=True)

In [240]:
E = E_m.groupby(['ano', 'sigla_uf'])['voters per municipality'].sum().reset_index(name = 'voters statewide')

In [241]:
LQ = df[['ano', 'id_municipio', 'sigla_uf', 'votes for congressperson on municipality', 'votes for deputies on municipality']]
LQ = LQ.merge(E_m[['ano', 'id_municipio', 'voters per municipality']], on = ['ano', 'id_municipio'], how = 'left')
LQ = LQ.merge(E, on = ['ano', 'sigla_uf'], how = 'left')

LQ['LQ'] = (LQ['votes for congressperson on municipality'] / LQ['votes for deputies on municipality']) * (LQ['voters statewide'] / LQ['voters per municipality'])

### Access to **geobr** data

In [242]:
query = """
SELECT id_municipio, sigla_uf, geometria
FROM basedosdados.br_geobr_mapas.municipio
WHERE sigla_uf = 'RJ'
"""

geometries = bd.read_sql(query, billing_project_id="diamundialdainfancia")

Downloading: 100%|██████████| 92/92 [00:01<00:00, 63.22rows/s]


In [243]:
map = geometries.merge(LQ, on = ['id_municipio', 'sigla_uf'], how = 'left')
map['geometria'] = map['geometria'].apply(wkt.loads)

In [244]:
map_gpd = gpd.GeoDataFrame(map, geometry = map['geometria'])

In [245]:
name = V_dm["nome"].iloc[0].replace(" ", "_").lower()

In [246]:
flt = map_gpd['ano'] == 2018

fig = px.choropleth(map_gpd[flt],
                    geojson = map_gpd[flt].geometry,
                    locations = map_gpd[flt].index,
                    color = 'LQ',
                    color_continuous_scale = 'Purples',
                    range_color=(map_gpd['LQ'].min(), map_gpd['LQ'].max()),
                    hover_data=['id_municipio'],
                    labels={'LQ': ''},
                    projection = 'mercator')

fig.update_layout(width = 900, height = 600,
                  margin = dict(l = 0, r = 150, t = 0, b = 0),
                  title = {
                        'y': 0.9,
                        'x': 0.5,
                        'xanchor': 'center',
                        'yanchor': 'top',
                        'font': {'size': 20},
                        'text': f'LQ - 2018 - {V_dm["nome"].iloc[0]}'
                  })

fig.update_geos(fitbounds = "locations", visible = False)
fig.show()

fig.write_image(f"viz/{name}-2018.png")

In [247]:
flt = map_gpd['ano'] == 2022

fig = px.choropleth(map_gpd[flt],
                    geojson = map_gpd[flt].geometry,
                    locations = map_gpd[flt].index,
                    color = 'LQ',
                    color_continuous_scale = 'Purples',
                    range_color=(map_gpd['LQ'].min(), map_gpd['LQ'].max()),
                    hover_data=['id_municipio'],
                    labels={'LQ': ''},
                    projection = 'mercator')

fig.update_layout(width = 900, height = 600,
                  margin = dict(l = 0, r = 150, t = 0, b = 0),
                  title = {
                        'y': 0.9,
                        'x': 0.5,
                        'xanchor': 'center',
                        'yanchor': 'top',
                        'font': {'size': 20},
                        'text': f'LQ - 2022 - {V_dm["nome"].iloc[0]}'
                  })

fig.update_geos(fitbounds = "locations", visible = False)
fig.show()

fig.write_image(f"viz/{name}-2022.png")